In [1]:
import torch, math, time, argparse, os
import random, dataset, utils, losses, net
import numpy as np

from dataset.market import Market
from dataset.build_market import build_market_dataloader
from net.os_net import get_embedding_model, osnet_ibn_x1_0, OSNet, OSBlock
from net.resnet import *
from net.googlenet import *
from net.bn_inception import *
from dataset import sampler
from eval import inference
from torch.utils.data.sampler import BatchSampler
from torch.utils.data.dataloader import default_collate

from tqdm import *
import datetime
import wandb

In [2]:
seed = 1
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [3]:
train_loader, val_dict, num_classes = build_market_dataloader()

=> Market1501 loaded
Dataset statistics:
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


In [4]:
model_name = 'osnet'

In [5]:
criterion = losses.Proxy_Anchor(nb_classes = num_classes, sz_embed = 512, mrg = 0.1, alpha = 32).cuda()

In [6]:
if model_name == 'resnet50':
    model = Resnet50(embedding_size=512, pretrained=True, is_norm=1, bn_freeze =1).cuda()

    param_groups = [
        {'params': list(set(model.parameters()).difference(set(model.model.embedding.parameters())))},
        {'params': model.model.embedding.parameters(), 'lr':float(1e-4) * 1},
    ]
    param_groups.append({'params': criterion.parameters(), 'lr':float(1e-4) * 100})

    modules = model.model.modules()
    for m in modules: 
        if isinstance(m, nn.BatchNorm2d):
            m.eval()
elif model_name == 'osnet':
    model = osnet_ibn_x1_0(pretrained=True).cuda()
    
    param_groups = [
        {'params': list(set(model.parameters()).difference(set(model.fc.parameters())))},
        {'params': model.fc.parameters(), 'lr':float(1e-4) * 1},
    ]
    param_groups.append({'params': criterion.parameters(), 'lr':float(1e-4) * 100})

    modules = model.modules()
    for m in modules: 
        if isinstance(m, nn.BatchNorm2d):
            m.eval()

Successfully loaded imagenet pretrained weights from "/home/nevin/.cache/torch/checkpoints/osnet_ibn_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [7]:
opt = torch.optim.AdamW(param_groups, lr=float(1e-4), weight_decay = 1e-4)

In [8]:
scheduler = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma = 0.5)

In [9]:
model_dir = 'models'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

In [10]:
def append_result(result, model):
    filename = f"{model_dir}/results_{current_datetime}.txt"

    # Open the file in append mode and write the result
    with open(filename, "a") as file:
        file.write('CMC\n')
        file.write(str(result[0]) + "\n")
        file.write('mAP\n')
        file.write(str(result[1]) + "\n\n")
    torch.save({'model_state_dict':model.state_dict()}, '{}/{}_{}_best.pth'.format(model_dir, model_name, current_datetime))

In [ ]:
best_mAP=0
current_datetime = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
for epoch in range(0, 600):
    model.train()

    losses_per_epoch = []
    if model == 'resnet50':
        unfreeze_model_param = list(model.model.embedding.parameters()) + list(criterion.parameters())
    else:
        unfreeze_model_param = list(model.fc.parameters()) + list(criterion.parameters())

    if epoch == 0:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = False
    if epoch == 1:
        for param in list(set(model.parameters()).difference(set(unfreeze_model_param))):
            param.requires_grad = True

    pbar = tqdm(enumerate(train_loader))

    for batch_idx, (x, y) in pbar:                         
        m = model(x.squeeze().cuda())
        loss = criterion(m, y.squeeze().cuda())
        
        opt.zero_grad()
        loss.backward()
        
        torch.nn.utils.clip_grad_value_(model.parameters(), 10)
        
        torch.nn.utils.clip_grad_value_(criterion.parameters(), 10)

        losses_per_epoch.append(loss.data.cpu().numpy())
        opt.step()

        pbar.set_description(
            'Train Epoch: {} [{}/{} ({:.0f}%)] Loss: {:.6f}'.format(
                epoch, batch_idx + 1, len(train_loader),
                100. * batch_idx / len(train_loader),
                loss.item()))
    scheduler.step()
    
    # if(epoch >= 0):
    #     with torch.no_grad():
    #         print("**Evaluating...**")
    #         Recalls = utils.evaluate_cos(model, val_dict['val_loader'])
    #     if best_recall[0] < Recalls[0]:
    #         best_recall = Recalls
    #         best_epoch = epoch
    #         if not os.path.exists(model_dir):
    #             os.makedirs(model_dir)
    #         torch.save({'model_state_dict':model.state_dict()}, '{}/{}_best.pth'.format(model_dir, model_name))
    #         with open('{}/{}_best_results.txt'.format(model_dir, model_name), 'w') as f:
    #             f.write('Best Epoch: {}\n'.format(best_epoch))
    #             for i in range(6):
    #                 f.write("Best Recall@{}: {:.4f}\n".format(2**i, best_recall[i] * 100))
    val_loader = val_dict['val_loader']
    num_query = val_dict['num_query']
    result = inference(model, val_loader, num_query)

    if result[1] > best_mAP:
        append_result(result, model)

0it [00:00, ?it/s]/home/nevin/anaconda3/envs/cv/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
Train Epoch: 0 [259/259 (100%)] Loss: 2.383882: : 259it [00:26,  9.85it/s]

Enter inferencing
Create evaluator


The test feature is normalized


/home/nevin/projects/assignments/cv_final_project/reid_metric.py:118: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/torch/csrc/utils/python_arg_parser.cpp:1519.)
  distmat.addmm_(1, -2, qf, gf.t())


Validation Results
mAP: 1.9%
CMC curve, Rank-1  :6.0%
CMC curve, Rank-2  :9.0%
CMC curve, Rank-4  :13.3%
CMC curve, Rank-6  :15.8%
CMC curve, Rank-8  :17.8%
CMC curve, Rank-10 :19.7%
CMC curve, Rank-12 :21.6%
CMC curve, Rank-14 :23.4%
CMC curve, Rank-16 :24.9%
CMC curve, Rank-18 :26.2%
CMC curve, Rank-20 :27.3%
CMC curve, Rank-22 :28.2%
CMC curve, Rank-24 :29.3%
CMC curve, Rank-26 :30.4%
CMC curve, Rank-28 :31.4%
CMC curve, Rank-30 :32.3%
CMC curve, Rank-32 :33.2%


...

The test feature is normalized
Validation Results
mAP: 62.2%
CMC curve, Rank-1  :83.9%
CMC curve, Rank-2  :89.0%
CMC curve, Rank-4  :92.8%
CMC curve, Rank-6  :94.7%
CMC curve, Rank-8  :95.6%
CMC curve, Rank-10 :96.1%
CMC curve, Rank-12 :96.4%
CMC curve, Rank-14 :96.9%
CMC curve, Rank-16 :97.2%
CMC curve, Rank-18 :97.7%
CMC curve, Rank-20 :97.7%
CMC curve, Rank-22 :98.0%
CMC curve, Rank-24 :98.1%
CMC curve, Rank-26 :98.1%
CMC curve, Rank-28 :98.2%
CMC curve, Rank-30 :98.4%
CMC curve, Rank-32 :98.4%


Train Epoch: 244 [259/259 (100%)] Loss: 0.047212: : 259it [01:14,  3.50it/s]

Enter inferencing
Create evaluator


The test feature is normalized
Validation Results
mAP: 62.2%
CMC curve, Rank-1  :83.6%
CMC curve, Rank-2  :88.9%
CMC curve, Rank-4  :92.8%
CMC curve, Rank-6  :94.7%
CMC curve, Rank-8  :95.7%
CMC curve, Rank-10 :96.1%
CMC curve, Rank-12 :96.5%
CMC curve, Rank-14 :96.7%
CMC curve, Rank-16 :97.1%
CMC curve, Rank-18 :97.5%
CMC curve, Rank-20 :97.7%
CMC curve, Rank-22 :97.9%
CMC curve, Rank-24 :98.0%
CMC curve, Rank-26 :98.2%
CMC curve, Rank-28 :98.2%
CMC curve, Rank-30 :98.3%
CMC curve, Rank-32 :98.4%


Train Epoch: 245 [108/259 (41%)] Loss: 0.048234: : 108it [00:30,  3.53it/s]

### Test codes